In [1]:
import requests

headers = {'Accept-Language': 'en'}
rawresponse = requests.get("https://tasks.hotosm.org/api/v1/project/search?campaignTag=Cyclone+Idai&mapperLevel=ALL", headers=headers)
mydict = rawresponse.json()


In [2]:
camps = mydict['results']
camps

[{'activeMappers': 0,
  'campaignTag': 'Cyclone Idai',
  'locale': 'en',
  'mapperLevel': 'INTERMEDIATE',
  'name': 'Cyclone Idai, Validation of #5879 & 5885: Buildings, Roads and Waterways',
  'organisationTag': 'Missing Maps',
  'percentMapped': 100.0,
  'percentValidated': 38.0,
  'priority': 'URGENT',
  'projectId': 5973,
  'shortDescription': '`VALIDATION ONLY PROJECT`\n\nHOT has activated to support the disaster response operations of several Missing Maps partners responding to the impact of Cyclone Idai in Mozambique. This project is to validate initial mapping in the area including and surrounding Biera.',
  'status': 'PUBLISHED'},
 {'activeMappers': 9,
  'campaignTag': 'Cyclone Idai',
  'locale': 'en',
  'mapperLevel': 'INTERMEDIATE',
  'name': 'Cyclone Idai, Mozambique: Dondo and Cidade Da Beira Roads and Waterways',
  'organisationTag': 'Missing Maps',
  'percentMapped': 79.0,
  'percentValidated': 1.0,
  'priority': 'URGENT',
  'projectId': 5950,
  'shortDescription': 'HOT 

In [3]:
med = {c['projectId']: c['percentMapped'] for c in camps}
vald = {c['projectId']: c['percentValidated'] for c in camps}
print(med)
print(vald)

{5973: 100.0, 5950: 79.0, 5969: 1.0, 5968: 1.0, 5946: 97.0, 5972: 100.0, 5965: 100.0, 5964: 100.0, 5932: 100.0, 5922: 100.0, 5916: 100.0, 5915: 100.0, 5908: 2.0, 5907: 100.0}
{5973: 38.0, 5950: 1.0, 5969: 0.0, 5968: 0.0, 5946: 8.0, 5972: 0.0, 5965: 0.0, 5964: 44.0, 5932: 0.0, 5922: 7.0, 5916: 43.0, 5915: 5.0, 5908: 0.0, 5907: 51.0}


In [10]:
import mwclient
import getpass

site = mwclient.Site(('https', 'wiki.openstreetmap.org'))
passy = getpass.getpass('Password:')
site.login('RAytoun', passy)
passy = "***"
page = site.Pages['Cyclone_Idai']
text = page.text()
#text

Password:········


In [11]:
# print out a report with the correct colour and status described
def formatReport(val):
    myval = int(val)
    myreport = ""
    if myval == 100:
        myreport = 'style="background:#A4FD9A" | Complete '
    else:
        myreport = 'style="background:#FDEB9A" | In Progress '
    return myreport + "(" + str(myval) + "%)"

# extract the relevant text
splitontable = text.split("HOT Activation Projects")
tablefront = splitontable[1].split("|}")
rawlines = tablefront[0].split('|-\n\n\n')

#print(rawlines)

# update the information within the table
updatedtext = ""
i = 0
while i < len(rawlines):
    updatedtext += "|-\n\n\n"
        
    # iterate through the lines and consider all rows which begin with hyperlinks
    if(rawlines[i].startswith("|[http")):
    
        # check if this is a record we need to update
        myid = int(rawlines[i].split(" ")[1].split("]")[0])
        if myid in med:
            cols = rawlines[i].split("||")
            mylen = len(cols)
            cols[mylen - 2] = formatReport(med[myid]) # mapped
            cols[mylen - 1] = formatReport(vald[myid]) # validated
            mynewline = " || ".join(cols) + "\n"
            updatedtext += mynewline
            print("********" + mynewline)

        # if not, we can just add the old info!    
        else:
            updatedtext += rawlines[i]
            print(rawlines[i])
    else:
        updatedtext += rawlines[i]

    i += 1

# reaggregating up
tablefront[0] = updatedtext
wholetable = " |} ".join(tablefront)
splitontable[1] = wholetable
fixedtext = " HOT Activation Projects\n".join(splitontable)
#print(fixedtext)

In [12]:
print(fixedtext)

{{languages}}<!--make the page name implicit so that links to versions of this page are not copied-->
{| style="background: #EEEEEE; border: 1px #aaa solid; color: black;" width="100%" cellpadding="6" 
|-
|style="background-color: #E0EEE0; border-bottom:1px solid #aaaaaa; font-size: 130%;" colspan="2"| '''General Information'''
|-
|
'''Cyclone Idai''' Response is an [https://www.hotosm.org/projects/CycloneIdai Activation of the Humanitarian OpenStreetMap Team (HOT)], requested by the International Federation of the Red Cross, to improve the basemap in areas of Mozambique affected by existing flooding in Zambezia province, exacerbated by the Cyclone, and Sofala province hit by the Cyclone itself, and resulting storm surges on the coast. HOT is also now supporting MSF in a number of project mapping areas of Zimbabwe impacted by Cyclone Idai.

[[File:Guekedou.png|thumb|500px|Map showing the Humanitarian Map Style]]
__TOC__
|}

<div style="clear:both; background:beige;box-shadow:3px 3px 2p

In [13]:
page.save(fixedtext,summary='auto-updated stats on relative task completion', minor=True)


OrderedDict([('result', 'Success'),
             ('pageid', 234598),
             ('title', 'Cyclone Idai'),
             ('contentmodel', 'wikitext'),
             ('oldrevid', 1849620),
             ('newrevid', 1850136),
             ('newtimestamp', '2019-05-07T20:02:17Z')])